In [1]:
from LSTM import LSTM
import DataPreparationLSTM
import os
import pandas as pd

2024-05-10 21:49:30.063280: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-10 21:49:30.147678: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-10 21:49:30.150353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 21:49:32.343025: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
input_data = pd.read_csv('data/SPY_data_5min.csv')
input_data['time'] = pd.to_datetime(input_data['time'])

# Filter trades within the time window from 09:30 to 16:00
input_data = input_data[(input_data['time'].dt.time >= pd.to_datetime('09:30').time()) & 
                 (input_data['time'].dt.time <= pd.to_datetime('16:00').time())]

input_data.drop(['Unnamed: 0','money','open','high','low'], axis=1, inplace=True)
input_data['time'] = pd.to_datetime(input_data['time']).dt.strftime('%H:%M')


In [3]:
learning_rate = 0.01
l_1 = 2
l_2 = 2
l_3 = 0
l_4 = 0


In [11]:
data = DataPreparationLSTM.DataPreparationLSTM(
        df = input_data,
        future = 1,
        lag = 20,
        min_max_scaler=True,
        log_transform=True,
        semi_variance=True,
        jump_detect=True,
        period_train= list(
            [
                pd.to_datetime("2006-01-01",format = "%Y-%m-%d"),
                pd.to_datetime("2010-12-31",format = "%Y-%m-%d"),
            ]
        ),
        period_validation = list(
            [
                pd.to_datetime("2011-01-01",format = "%Y-%m-%d"),
                pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),  
            ]
        ),
        period_test= list(
            [
                pd.to_datetime("2016-01-01",format = "%Y-%m-%d"),
                pd.to_datetime("2018-12-31",format = "%Y-%m-%d"),
            ]
        ),
    )

In [12]:
data.data_transformation()

In [13]:
data.df

,RV,RVp,RVn,BV
date,,,,
2004-01-02,0.000048,0.000022,0.000026,0.000036
2004-01-05,0.000023,0.000014,0.000009,0.000026
2004-01-06,0.000021,0.000011,0.000009,0.000018
2004-01-07,0.000029,0.000017,0.000012,0.000025
2004-01-08,0.000033,0.000015,0.000018,0.000034
...,...,...,...,...
2020-09-28,0.000059,0.000029,0.000030,0.000061
2020-09-29,0.000057,0.000023,0.000034,0.000052
2020-09-30,0.000137,0.000065,0.000072,0.000114


In [15]:
data.generate_training_test_split()

In [16]:
data.training_set

,Target,RV,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,...,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,RVn
date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,-10.405012,-9.763411,-11.075989,-11.096658,-11.306916,-10.450167,-11.808040,-11.359726,-10.542782,-10.596730,...,-10.690514,-10.735768,-10.587664,-10.944276,-10.689253,-10.051101,-10.382023,-10.535827,-10.984579,-11.211248
2006-01-04,-10.536146,-10.405012,-9.763411,-11.075989,-11.096658,-11.306916,-10.450167,-11.808040,-11.359726,-10.542782,...,-11.144290,-10.690514,-10.735768,-10.587664,-10.944276,-10.689253,-10.051101,-10.382023,-10.535827,-11.274882
2006-01-05,-10.549400,-10.536146,-10.405012,-9.763411,-11.075989,-11.096658,-11.306916,-10.450167,-11.808040,-11.359726,...,-10.875784,-11.144290,-10.690514,-10.735768,-10.587664,-10.944276,-10.689253,-10.051101,-10.382023,-11.350369
2006-01-06,-11.087727,-10.549400,-10.536146,-10.405012,-9.763411,-11.075989,-11.096658,-11.306916,-10.450167,-11.808040,...,-10.596730,-10.875784,-11.144290,-10.690514,-10.735768,-10.587664,-10.944276,-10.689253,-10.051101,-11.283140
2006-01-09,-10.940091,-11.087727,-10.549400,-10.536146,-10.405012,-9.763411,-11.075989,-11.096658,-11.306916,-10.450167,...,-10.542782,-10.596730,-10.875784,-11.144290,-10.690514,-10.735768,-10.587664,-10.944276,-10.689253,-12.112493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-27,-12.005176,-11.955387,-11.696206,-11.944330,-12.127789,-11.241269,-11.258388,-10.260722,-10.655160,-10.907654,...,-10.658113,-10.366188,-9.924883,-10.925525,-10.456450,-10.676548,-10.295250,-9.670396,-9.390921,-13.301178
2010-12-28,-12.410654,-12.005176,-11.955387,-11.696206,-11.944330,-12.127789,-11.241269,-11.258388,-10.260722,-10.655160,...,-11.080212,-10.658113,-10.366188,-9.924883,-10.925525,-10.456450,-10.676548,-10.295250,-9.670396,-12.414662
2010-12-29,-11.493342,-12.410654,-12.005176,-11.955387,-11.696206,-11.944330,-12.127789,-11.241269,-11.258388,-10.260722,...,-10.693791,-11.080212,-10.658113,-10.366188,-9.924883,-10.925525,-10.456450,-10.676548,-10.295250,-12.918003


In [17]:
data.training_set.describe()

,Target,RV,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,...,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,RVn
count,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,...,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000,1216.000000
mean,-9.698261,-9.697124,-9.697019,-9.696693,-9.695785,-9.694507,-9.694385,-9.694109,-9.692956,-9.691697,...,-9.691656,-9.691722,-9.691459,-9.691665,-9.691344,-9.690845,-9.690858,-9.691360,-9.691409,-10.440380
std,1.097419,1.096634,1.096496,1.096020,1.094226,1.092432,1.092189,1.091724,1.090085,1.088153,...,1.087900,1.087961,1.087705,1.087919,1.087566,1.087261,1.087269,1.087519,1.087575,1.159854
min,-12.410654,-12.410654,-12.410654,-12.410654,-12.144916,-12.144916,-12.144916,-12.144916,-12.144916,-12.144916,...,-12.144916,-12.144916,-12.144916,-12.144916,-12.144916,-12.144916,-12.144916,-12.144916,-12.144916,-13.301178
25%,-10.541804,-10.537294,-10.537294,-10.537294,-10.537294,-10.532207,-10.532207,-10.532207,-10.532207,-10.532207,...,-10.537294,-10.537294,-10.537294,-10.537294,-10.537294,-10.532207,-10.532207,-10.535906,-10.535906,-11.294375
50%,-9.792959,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,...,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-9.791912,-10.541335
75%,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,...,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.042806,-9.694593
max,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,...,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-5.898562,-6.407458


In [18]:
data.generate_validation_split()

In [19]:
data.y_train

array([[0.30798735],
       [0.28785043],
       [0.28581515],
       ...,
       [0.14086285],
       [0.1854139 ],
       [0.19424906]])

In [20]:
data.generate_splits()

In [21]:
ovo =pd.DataFrame(data.back_transformation(data.y_train))
ovo

,0
0,0.000030
1,0.000027
2,0.000026
3,0.000015
4,0.000018
...,...
1211,0.000006
1212,0.000004
1213,0.000010
1214,0.000014


In [22]:
data.x_train.shape

(1216, 21, 1)

In [23]:
data.y_train

array([[0.30798735],
       [0.28785043],
       [0.28581515],
       ...,
       [0.14086285],
       [0.1854139 ],
       [0.19424906]])

In [24]:
data.x_test.shape

(742, 21, 1)

In [25]:
splits = data.splits

In [26]:
len(splits[0][0])

1216

In [27]:
len(splits[1][0])

1626

In [28]:
errors = []
folds =  data.splits

In [29]:

for (train_x,train_y, validate_x, validate_y) in folds:

    model = LSTM(
        #training_set= train_x +train_y,
        #testing_set= validate_x+validate_y,
        train_matrix= train_x,
        train_y = train_y,
        test_matrix= validate_x,
        test_y=validate_y,
        epochs= 20,
        learning_rate= learning_rate,
        layer_one=l_1,
        layer_two= l_2,
        layer_three=l_3,
        layer_four= l_4

    )
    model.train_lstm()
    model.predict_lstm()
    
    prediction_train = model.prediction_train
    prediction_test = model.prediction_test

    target_scaler = data.applied_scaler_targets

    prediction_train_normal = data.back_transformation(prediction_train)
    prediction_test_normal = data.back_transformation(prediction_test)

    train_y_normal = data.back_transformation(model.train_y)
    test_y_normal = data.back_transformation(model.test_y)

    model.prediction_train = prediction_train_normal
    model.prediction_test = prediction_test_normal
    model.test_y = test_y_normal
    model.train_y = train_y_normal
    model.make_accuracy_measures()
    
    errors.append([model.train_accuracy, model.test_accuracy])

Epoch 1/20
38/38 [==============================] - 4s 33ms/step - loss: 0.0200 - val_loss: 0.0093
Epoch 2/20
38/38 [==============================] - 1s 14ms/step - loss: 0.0067 - val_loss: 0.0032
Epoch 3/20
38/38 [==============================] - 0s 13ms/step - loss: 0.0047 - val_loss: 0.0036
Epoch 4/20
38/38 [==============================] - 0s 13ms/step - loss: 0.0046 - val_loss: 0.0030
Epoch 5/20
38/38 [==============================] - 0s 13ms/step - loss: 0.0044 - val_loss: 0.0029
Epoch 6/20
38/38 [==============================] - 0s 11ms/step - loss: 0.0042 - val_loss: 0.0030
Epoch 7/20
38/38 [==============================] - 0s 12ms/step - loss: 0.0041 - val_loss: 0.0029
Epoch 8/20
38/38 [==============================] - 1s 14ms/step - loss: 0.0042 - val_loss: 0.0028
Epoch 9/20
38/38 [==============================] - 0s 13ms/step - loss: 0.0040 - val_loss: 0.0035
Epoch 10/20
38/38 [==============================] - 1s 14ms/step - loss: 0.0040 - val_loss: 0.0028
Epoch 11/

In [30]:
for e_train,e_test in errors:
    print('Trening' + str(e_train))
    print('Test '+ str(e_test))
    print('-------------------------------------------------------')

Trening{'MSE': 2.2423030076392133e-08, 'MAE': 5.387414408407312e-05, 'RSquared': 0.6347441641280319}
Test {'MSE': 5.25836986310733e-10, 'MAE': 1.5786679610186118e-05, 'RSquared': 0.4107190130695183}
-------------------------------------------------------
Trening{'MSE': 2.1911312038501995e-08, 'MAE': 4.748376419449802e-05, 'RSquared': 0.5443849008013979}
Test {'MSE': 2.950936786168569e-10, 'MAE': 1.2807295967342088e-05, 'RSquared': 0.08024813375646578}
-------------------------------------------------------
Trening{'MSE': 1.2196124544762414e-08, 'MAE': 3.6810389513266607e-05, 'RSquared': 0.7192293396765294}
Test {'MSE': 2.458424228356317e-10, 'MAE': 1.1373065177950039e-05, 'RSquared': 0.12356966309642081}
-------------------------------------------------------
Trening{'MSE': 1.3626772936689454e-08, 'MAE': 4.461201020595963e-05, 'RSquared': 0.6563688861502031}
Test {'MSE': 4.5405983030563057e-10, 'MAE': 1.5354961245525776e-05, 'RSquared': 0.3603818003200132}
-----------------------------

In [31]:
from sklearn import metrics

In [32]:
data.y_train.shape

(1216, 1)

In [33]:
data.y_val.shape

(1222, 1)

In [34]:
data.y_test.shape

(742, 1)

In [35]:
model.prediction_test.shape

(203, 1)

In [36]:
model.prediction_train.shape

(2235, 1)